# Kurdish Letter OCR - CNN Model

## Overview
This notebook trains and evaluates a Convolutional Neural Network (CNN) to classify Kurdish letters (EEE, LLL, OOO, RRR, VVV) from grayscale images.

## Pipeline
1. **Configuration** - Set up parameters and load dependencies
2. **Model Architecture** - Define CNN structure
3. **Load Data** - Read and preprocess images from folders
4. **Data Preparation** - Normalize, split into train/test sets
5. **Training** - Train model for 20 epochs
6. **Evaluation** - Test accuracy on unseen data

## Key Features
- **Input**: 64x64 grayscale images
- **Output**: 5 Kurdish letter classes
- **Model**: 3-layer CNN with pooling and fully connected layers
- **Optimizer**: Adam with learning rate 0.001
- **Device**: GPU support (falls back to CPU)

## Expected Results
- Training accuracy increases each epoch
- Final test accuracy typically 85-95% depending on data quality
- Model weights saved for later inference

## Libraries required


In [ ]:
import os                                    # File and directory operations
import cv2                                   # OpenCV - Image reading, resizing, processing
import numpy as np                           # NumPy - Array operations and numerical computing
import torch                                 # PyTorch - Deep learning framework
import torch.nn as nn                        # PyTorch neural network modules
import torch.optim as optim                  # PyTorch optimization algorithms (Adam, SGD, etc.)
from torch.utils.data import DataLoader, TensorDataset  # Data loading utilities
from sklearn.model_selection import train_test_split    # Train/test data splitting
from sklearn.preprocessing import LabelEncoder          # Categorical label encoding

## Configuration

Sets up all necessary parameters, imports, and directories for the Kurdish letter classification model.

**Parameters:**
- `IMG_SIZE`: 64x64 pixels (image dimensions)
- `CHANNELS`: 1 (grayscale images)
- `EPOCHS`: 20 training iterations
- `BATCH_SIZE`: 32 images per batch
- `LEARNING_RATE`: 0.001 for Adam optimizer

**Categories:** 5 Kurdish letters (EEE, LLL, OOO, RRR, VVV)

**Device:** Uses GPU if available, otherwise CPU

In [ ]:
IMG_SIZE = 64
CHANNELS = 1
EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 0.001
DATA_DIR = "."
MODEL_PATH = "kurdish_letter_model_pytorch.pth"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Categories in alphabetical order (for consistent label encoding)
CATEGORIES = [
    {"folder": "EEE_letters", "prefix": "eee_letter_"},
    {"folder": "LLL_letters", "prefix": "lll_letter_"},
    {"folder": "OOO_letters", "prefix": "ooo_letter_"},
    {"folder": "RRR_letters", "prefix": "rrr_letter_"},
    {"folder": "VVV_letters", "prefix": "vvv_letter_"}
]

EXTENSIONS = [".png", ".jpg", ".jpeg", ".bmp"]

## Model Architecture

Defines the KurdishCNN - a Convolutional Neural Network for image classification.

**Architecture:**
- **Input**: 1x64x64 grayscale images
- **Convolutional Layers**: 3 layers with ReLU activation
  - Conv1: 32 filters
  - Conv2: 64 filters
  - Conv3: 64 filters
- **Pooling**: MaxPool2d (2x2) after each convolution
- **Fully Connected Layers**:
  - FC1: Flattened features → 64 neurons (with dropout)
  - FC2: 64 → 5 classes (Kurdish letters)

**Total parameters**: ~500K trainable weights

In [ ]:
class KurdishCNN(nn.Module):
    def __init__(self, num_classes):
        super(KurdishCNN, self).__init__()

        # Convolutional Layers
        # Input: (1, IMG_SIZE, IMG_SIZE)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.relu = nn.ReLU()

        # Dynamically calculate the input features for the linear layer.
        # Pass a dummy tensor through the conv layers to get the output size.
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, IMG_SIZE, IMG_SIZE)
            x = self.pool(self.relu(self.conv1(dummy_input)))
            x = self.pool(self.relu(self.conv2(x)))
            x = self.pool(self.relu(self.conv3(x)))
            num_flat_features = x.numel() // x.shape[0] # x.shape[0] is batch size 1

        # Fully Connected Layers
        self.fc1 = nn.Linear(num_flat_features, 64)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))

        x = x.view(x.size(0), -1) # Flatten the tensor dynamically
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

## Loading Data

Loads and preprocesses all images from the letter category folders.

**Process:**
1. Scans all "_letters" folders
2. Reads images in supported formats (PNG, JPG, JPEG, BMP)
3. Converts to grayscale
4. Resizes to 64x64 pixels
5. Stores image arrays and corresponding labels

**Output:**
- `X`: Image array data
- `y`: Corresponding class labels (folder names)
- Total images loaded across all 5 categories

In [ ]:
def load_data():
    """Load and preprocess images from category folders."""
    data = []
    labels = []

    print("Loading and Preprocessing Images...")

    for category_idx, cat in enumerate(CATEGORIES):
        folder_path = os.path.join(DATA_DIR, cat["folder"])

        if not os.path.exists(folder_path):
            print(f"Warning: Folder '{folder_path}' not found. Skipping.")
            continue

        count = 0
        # Load all image files in the folder
        for filename in os.listdir(folder_path):
            # Check if file has a supported image extension
            if not any(filename.lower().endswith(ext) for ext in EXTENSIONS):
                continue

            image_path = os.path.join(folder_path, filename)

            # Skip if it's a directory
            if not os.path.isfile(image_path):
                continue

            try:
                img_array = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                if img_array is None:
                    continue

                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append(new_array)
                labels.append(cat["folder"])
                count += 1

            except Exception as e:
                print(f"  Error processing {image_path}: {e}")

        print(f"Loaded {count} images for class: {cat['folder']}")

    return np.array(data), np.array(labels)

## Data Preparation

Preprocesses the loaded images and prepares them for training and testing.

**Preprocessing Steps:**
1. **Normalization** - Scale pixel values from [0-255] to [0-1] by dividing by 255
2. **Reshaping** - Reshape data to (N, IMG_SIZE, IMG_SIZE, CHANNELS) format
3. **Transpose** - Reorder dimensions to PyTorch format (N, CHANNELS, IMG_SIZE, IMG_SIZE)
4. **Tensor Conversion** - Convert NumPy arrays to PyTorch tensors (float32 for images, long for labels)

**Label Encoding:**
- Converts categorical class names to numeric labels:
  - EEE_letters → 0
  - LLL_letters → 1
  - OOO_letters → 2
  - RRR_letters → 3
  - VVV_letters → 4

**Data Splitting:**
- Train/Test split: 80% training, 20% testing
- Creates DataLoaders for batch processing
- Batch size: 32 images per batch
- Shuffles training data for better learning

**Output Shapes:**
- Training set: (N_train, 1, 64, 64)
- Test set: (N_test, 1, 64, 64)

In [ ]:
# --- DATA PREPARATION ---
X, y = load_data()

if len(X) == 0:
    print("No images loaded. Exiting.")
    exit()

# Normalize and reshape for PyTorch (N, 1, 64, 64)
X = X / 255.0
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, CHANNELS)
X = np.transpose(X, (0, 3, 1, 2))

X_tensor = torch.tensor(X, dtype=torch.float32)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

print(f"Classes found: {le.classes_}")

# Split and create data loaders
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training shape: {X_train.shape}")
print(f"Testing shape: {X_test.shape}")

# --- MODEL INITIALIZATION ---
model = KurdishCNN(num_classes=len(CATEGORIES)).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

## Training & Evaluation

Trains the CNN model on the training dataset and evaluates performance.

**Training Process:**
1. Creates and initializes the model
2. Sets up loss function (CrossEntropyLoss) and optimizer (Adam)
3. For each of 20 epochs:
   - Forward pass through model
   - Calculate loss on training batch
   - Backward pass (backpropagation)
   - Update weights via gradient descent
   - Track training loss and accuracy
4. Evaluates model on test set (20% of data)
5. Saves trained model weights to file

**Metrics:**
- Displays loss and accuracy per epoch
- Final test accuracy on unseen data
- Model saved as `kurdish_letter_model_pytorch.pth`

In [ ]:
# --- TRAINING LOOP ---
print("\nStarting Training...")

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {epoch_acc:.2f}%")

# --- EVALUATION ---
print("\nEvaluating on Test Set...")
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Final Test Accuracy: {100 * correct / total:.2f}%")

# --- SAVE MODEL ---
torch.save(model.state_dict(), MODEL_PATH)
print(f"Model saved as '{MODEL_PATH}'")